<a href="https://colab.research.google.com/github/AgnTR/ImersaoIA_Alura_Google/blob/main/Imersao_IA_Alura_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validação de Produto/Serviço com Personas usando IA
Objetivo: criar um modelo de IA que gere personas e com base em suas características e personalidades emita opinião sobre determinado produto/serviço para que seja emitida uma avaliação do produto/serviço e possíveis melhorias.

Autor: Agnaldo Teodoro Rodrigues

Instalação das bibliotecas necessárias

In [15]:
!pip install -q -U google-generativeai

Importação das bibliotecas necessárias

In [16]:
from IPython.display import clear_output
import google.generativeai as genai
from google.colab import userdata
import pandas as pd

Funçoes criadas

In [17]:
def criar_lista_personas():
  """
  Função para criar uma lista de personas com nome da persona e sua personalidades, não permitindo nomes duplicados.

  Retorna:
    Uma lista de dicionários com as informações coletadas do usuário.
  """
  lista_personas = []
  while True:
    nome = input("Digite o nome da persona: ")
    if nome == "":
      break
    if any(persona["nome"] == nome for persona in lista_personas):
      print(f"O nome '{nome}' já foi adicionado. Tente outro nome.")
      continue
    personalidade = input(f"Digite a personalidade de {nome}: ")
    nova_persona = {"nome": nome, "personalidade": personalidade}
    lista_personas.append(nova_persona)
    clear_output()  # Limpa a saída anterior
  return lista_personas

In [18]:
def exibir_informacoes(personas):
  """
  Função para percorrer a lista de personas criada.

  Argumento:
    personas: lista de personas criada.
  """
  for indice, persona in enumerate(personas):
    nome = persona["nome"]
    personalidade = persona["personalidade"]
    print(f"{indice + 1}. Nome: {nome}")
    print(f"  Personalidade: {personalidade}")
    print()  # Adiciona uma linha em branco entre cada persona

In [19]:
def gera_categorizacao(resultado_pesquisa):
  """
  Função para gerar a categorização da resposta das personas.
  O objetivo aqui não é identificar a persona e a categoria de sua resposta, é apenas categorizar a resposta
    para se emitir uma avaliação posterior da pesquisa

  Argumentos:
    resultado_pesquisa: contém a opinião das personas sobre o produto/serviço pesquisado

  Retorna:
    Lista a categoria da resposta
  """

  # aqui são usadas as variáveis de configuração novamente para caso haja a necessidade de mudança nelas somente
  # para esta função.

  #configurações gerais
  generation_config = {
    "candidate_count": 1,
    "temperature": 0.9
  }

  #configurações de segurança
  safety_settings = {
    "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
    "HATE": "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE"
  }

  model = genai.GenerativeModel(
    model_name = "gemini-1.5-pro-latest",
    generation_config = generation_config,
    safety_settings = safety_settings
  )

  resultado = []
  for resposta in resultado_pesquisa:
    response = model.generate_content('Classifique a seguinte opinião retornando somente se foi boa, média ou ruim: ' + resposta["resposta"])
    resultado.append(response.text)

  return resultado

Usando a chave

In [26]:
user_key = userdata.get('MY_KEY')
genai.configure(api_key = user_key)

Realizando configurações gerais

In [21]:
#configurações gerais
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}

#configurações de segurança
safety_settings = {
    "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
    "HATE": "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE"
}

Criando a lista de personas.<br>
Aqui o usuário poderá criar suas próprias personas ou caso ele queira pode importar uma lista de dicionário de personas com as chaves "nome"e "personalidade".<br>
Quanto mais detalhar a personalidade melhor será para o modelo emitir uma opinião sobre o produto/serviço que se quer avaliar.<br>
Obs: para encerrar a entrada de dados de personas basta deixar o campo nome em branco e teclar ENTER.

In [ ]:
personas = criar_lista_personas()

Digite o nome da persona: 


Listagem para conferência das personas criadas

In [22]:
exibir_informacoes(personas)

1. Nome: André
  Personalidade: Gosta de futebol e trabalha numa farmácia. É ambicioso e estuda muito.

2. Nome: José
  Personalidade: Cabeleireiro que sonha ter seu próprio salão para atender pessoas famosas.

3. Nome: Maria
  Personalidade: Mulher dedicada ao trabalho e cuidar da família, gosta de esportes radicais e ama fazer trilha.

4. Nome: Geraldo
  Personalidade: Garçom que está cursando faculdade de direito e quer se tornar juíz.

5. Nome: Márcia
  Personalidade: Empresária do ramo da moda, ama viajar e gosta de curtir a vida com leveza.



Aqui é definida a pergunta da pesquisa com as personas. A pergunta será sobre um determinado produto ou serviço.

In [ ]:
pergunta = input('Informe a pergunta para o produto/serviço: ')

Informe a pergunta para o produto/serviço: Bicicleta estilosa para passeio com a família, leve e muito bonita.


Gerando as opiniões das personas para a pergunta estipulada.<br>
OBS: FOI REALIZADA A TENTATIVA DE RODAR NOVAMENTE O MODELO COM NOVAS PERSONAS E MELHORIA NA GERAÇÃO DA RESPOSTA ONDE ELES RESPONDEM COMO CONSUMIDORES DO PRODUTO/SERVIÇO MESMO, PORÉM DEU ERRO DE EXCESSO DE QUOTA. OS RESULTADO DAQUI PARA FRENTE SÃO DA EXECUÇÃO ANTERIOR DO MODELO, COM A MESMA PERGUNTA PORÉM COM UMA LISTA DE PERSONAS UM POUCO DIFERENTE APENAS.

In [27]:
opinioes = []
for persona in personas:
  nome = persona["nome"]

  model = genai.GenerativeModel(
      model_name = "gemini-1.5-pro-latest",
      generation_config = generation_config,
      safety_settings = safety_settings,
      system_instruction = "Responda com se fosse uma pessoa cosumidora com a seguinte personalidade: " + persona["personalidade"]
  )

  resposta = model.generate_content(pergunta).text

  opiniao = {"nome": nome, "resposta": resposta}
  opinioes.append(opiniao)

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

Listagem das opiniões dadas sobre a pesquisa

In [ ]:
opinioes

[{'nome': 'Mário',
  'resposta': 'Cara, preciso de uma bike nova! Essa que tenho tá detonada, preciso de algo estiloso pra andar com a galera e com a família. Tava pensando numa bike leve, pra aguentar umas trilhas leves, sabe? Mas que também seja bonita pra usar na cidade. Me recomenda alguma? Ah, e de preferência que não custe um rim, né? 😅💸  Tô investindo na minha futura clínica! 😂 \n'},
 {'nome': 'Márcia',
  'resposta': 'Aiiii que tudo! 😍 Adoro! Me fala, você prefere algo mais vintage, retrô, ou moderna? 🤔  Temos cada modelo lindooo! 💖 Perfeitas para um passeio no calçadão da praia! 🌴☀️ Imagina só, você, a família, sentindo a brisa do mar... 🌊 É tudo de bom! 🙌 Me conta mais sobre o que você busca que te mostro as melhores opções! 😉 \n'},
 {'nome': 'Paula',
  'resposta': 'Ai, meu bem, você quer uma bicicleta estilosa para passear com a família? Que ideia maravilhosa! Me lembra aqueles filmes coreanos, sabe? Família feliz pedalando em um dia de sol... 🥰 \n\nMas me fale mais, meu anjo

Aqui é feita a categorização das resposta pelo Gemini como boa, média ou ruim.

In [ ]:
model = genai.GenerativeModel(
  model_name = "gemini-1.5-pro-latest",
  generation_config = generation_config,
  safety_settings = safety_settings
)

resposta_categorizada = []
for resposta in opinioes:
  response = model.generate_content('Classifique a seguinte opinião retornando somente se foi boa, média ou ruim: ' + resposta["resposta"])
  resposta_categorizada.append(response.text)

Listagem das respostas categorizadas.<br>
Não é necessária a identificação o objetivo é apenas saber no geral se o produto/serviço será bem aceito.

In [ ]:
resposta_categorizada

['Média. \n', 'Boa \n', 'Boa \n', 'Boa \n']

Melhoria na apresentação do resultado da pesquisa

In [ ]:
resultado_pesquisa = pd.Series(resposta_categorizada)

In [ ]:
# contato a ocorrência de cada valor
value_counts = resultado_pesquisa.value_counts()

In [ ]:
# convertendo em porcentagens
percentages = value_counts / resultado_pesquisa.size * 100

In [ ]:
# criando um dataframe
df = pd.DataFrame({'Resultado': value_counts.index, 'Contagem': value_counts.values, 'Porcentagem': percentages.values})

In [ ]:
df

,Resultado,Contagem,Porcentagem
0,Boa \n,3,75.0
1,Média. \n,1,25.0


Com base nas opiniões é solicitado ao modelo uma análise do produto.

In [ ]:
df_respostas = pd.DataFrame(opinioes)

In [ ]:
somente_respostas = df_respostas['resposta'].tolist()

In [ ]:
somente_respostas

['Cara, preciso de uma bike nova! Essa que tenho tá detonada, preciso de algo estiloso pra andar com a galera e com a família. Tava pensando numa bike leve, pra aguentar umas trilhas leves, sabe? Mas que também seja bonita pra usar na cidade. Me recomenda alguma? Ah, e de preferência que não custe um rim, né? 😅💸  Tô investindo na minha futura clínica! 😂 \n',
 'Aiiii que tudo! 😍 Adoro! Me fala, você prefere algo mais vintage, retrô, ou moderna? 🤔  Temos cada modelo lindooo! 💖 Perfeitas para um passeio no calçadão da praia! 🌴☀️ Imagina só, você, a família, sentindo a brisa do mar... 🌊 É tudo de bom! 🙌 Me conta mais sobre o que você busca que te mostro as melhores opções! 😉 \n',
 'Ai, meu bem, você quer uma bicicleta estilosa para passear com a família? Que ideia maravilhosa! Me lembra aqueles filmes coreanos, sabe? Família feliz pedalando em um dia de sol... 🥰 \n\nMas me fale mais, meu anjo! Você prefere cores vibrantes ou algo mais clássico? E o tamanho? Precisa ser adequada para todos,

In [ ]:
model = genai.GenerativeModel(
    model_name = "gemini-1.5-pro-latest",
    generation_config = generation_config,
    safety_settings = safety_settings,
    system_instruction = "Responda com se fosse um grande analista de mercado e produto"
)

analise = model.generate_content("Analise todas as opinioes contidas na lista: " + ", ".join(somente_respostas) + " e veja o que poderia melhorar no produto e estabeleça uma campanha de marketing").text

Resultado da análise do produto

In [ ]:
print(analise)

## Análise das Opiniões e Sugestões para Campanha de Marketing de Bicicletas

As opiniões coletadas revelam um público-alvo que busca bicicletas com as seguintes características:

**Desejos do Consumidor:**

* **Estilo:**  A bicicleta precisa ser "estilosa",  indicando um público que se importa com a aparência do produto e como ele os representa socialmente. 
* **Versatilidade:** Há interesse em bicicletas que sirvam tanto para trilhas leves quanto para uso urbano, evidenciando um desejo por um produto multifuncional.
* **Leveza:**  A busca por bicicletas leves sugere a necessidade de um produto que proporcione conforto e facilidade no transporte.
* **Preço Acessível:** A menção a "não custar um rim" demonstra a importância de um bom custo-benefício.
* **Uso Familiar:**  As respostas realçam o uso da bicicleta para momentos em família, indicando a necessidade de modelos que atendam diferentes idades e tamanhos.

**Pontos Fracos nas Abordagens:**

* **Falta de foco em especificações:** 